In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from sys import stdout
import os, glob
from scipy.signal import detrend
from scipy import arctan2
import functions as f

In [3]:
import matplotlib.pyplot as plt

In [4]:
path_to_csvs   = "/Volumes/pdoubraw-1/repos/jet/data/TTU 200 m tower data/"
path_to_csvs   = "C:\\Users\\krydalch\\Desktop\\MetMastData\\TTU tall tower\\"
filepaths_dict = f.get_file_paths(path_to_csvs, day=17)

In [5]:
height_feet_dict, height_meters_dict, header_names_dict, unique_heights, unique_heights_all = f.get_file_headers(path_to_csvs)

In [6]:
u = {}
v = {}
w = {}

In [7]:
year   = 2017 
month  = 2
day    = 17
hour   = 14    
minute = 40

In [8]:


desired_start = pd.Timestamp(year=year,month=month,day=day,hour=hour,minute=minute)
desired_end   = desired_start + pd.Timedelta('10min')
timestamp_in_filename = desired_start.floor('30min')

filenames = sorted(glob.glob(os.path.join(path_to_csvs,'*{0:%Y%m%d}_T{0:%H%M}*'.format(timestamp_in_filename))))

perturbations = {}


RotMatrix, boom_azm_deg, boom_vert_deg = f.get_boom_tower_angles(path_to_csvs)

for filepath in filenames:

    group = int(os.path.split(filepath)[-1].split('TR_D')[-1].split('.csv')[0])

    print(filepath)

    df           = f.csv_to_df(filepath, header_names_dict[group])  
    subdf        = f.subset_df(df)
    df_dict      = f.df_to_df_dict(subdf)

    df_dict_rot  = f.rotate_df_dict(df_dict,RotMatrix)
    df_dict_pert = f.compute_perturbations(df_dict_rot)

    for height in df_dict_pert.keys():

        perturbations[height] = df_dict_pert[height].loc[desired_start:desired_end].copy()

desired_index = pd.DatetimeIndex(start=desired_start, end=desired_end, freq='20ms').tz_localize('UTC')

for height in perturbations.keys():
    perturbations[height] = perturbations[height].reindex(desired_index).interpolate(method='index').iloc[0:30000].copy()

u[minute] = perturbations[382.0].U.copy()
v[minute] = perturbations[382.0].V.copy()
w[minute] = perturbations[382.0].V.copy()
    
    

C:\Users\krydalch\Desktop\MetMastData\TTU tall tower\FT2_E07_C04_R27885_D20170217_T1430_TR_D01.csv


TypeError: cannot compare a dtyped [float64] array with a scalar of type [bool]

In [ ]:
fig = plt.figure(figsize=(12,4))
ax  = fig.add_subplot(111)

for key in u.keys():
    utmp = u[key].resample('1s').mean().copy()
    t    = (utmp.index - utmp.index[0]).total_seconds()
    utmp = utmp.values
    
    ax.plot(t, utmp, label="{0:02d} - {1:02d}".format(key,key+10))
    
plt.legend()
ax.set_title("Hour {0}".format(hour))
ax.set_ylabel("U [m/s]")
ax.set_xlabel("time [s]")
fig.savefig("/Volumes/pdoubraw-1/repos/jet/figs/sonic_U_ts_hour_{0}.png".format(hour))